### Tekoälyn käytön kuvaus työn suorittamisessa

ChatGPT:tä käytettiin työn aikana tukityökaluna raportin rakenteen jäsentämisessä, tekstin muokkaamisessa akateemiseen tyyliin sekä suomenkielisen ilmaisun tarkentamisessa.

Tekoälyä hyödynnettiin myös lineaarisen ja logistisen regression käsitteiden selventämisessä sekä mallien tulosten ja kertoimien tulkinnan tukena. Mallien rakentaminen, aineiston esikäsittely, koulutus- ja testijoukon muodostaminen sekä tulosten laskenta toteutettiin työn tekijän toimesta kurssimateriaalien ja ohjeiden pohjalta.

Työ eteni vuorovaikutteisessa prosessissa, jossa ChatGPT toimi keskustelun ja ajattelun tukena. Tekoäly auttoi hahmottamaan analyysin eri vaiheita ja niiden välisiä yhteyksiä, mutta työn keskeiset ratkaisut ja toteutus perustuivat tekijän omaan työskentelyyn.

# 1. Business Understanding

Tämän työn tavoitteena on hyödyntää lineaarista ja logistista regressiota kiinteistöjen hintojen mallintamiseen. Lineaarista regressiota käytetään neliömetrihintojen ennustamiseen, kun taas logistista regressiota sovelletaan kiinteistöjen luokitteluun sen perusteella, ylittääkö hinta keskiarvon vai ei.

# 2. Data Understanding

Tässä vaiheessa ladataan aineisto ja suoritetaan sen alustava analyysi. Vaiheen tavoitteena on tutkia aineiston rakennetta, muuttujien tyyppejä, arvojen vaihteluvälejä sekä mahdollisia ongelmia, kuten puuttuvia arvoja tai virheellisiä tietotyyppejä.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, accuracy_score, confusion_matrix

df = pd.read_csv('https://archive.ics.uci.edu/static/public/477/data.csv')
df

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1
...,...,...,...,...,...,...,...,...
409,410,2013.000,13.7,4082.01500,0,24.94155,121.50381,15.4
410,411,2012.667,5.6,90.45606,9,24.97433,121.54310,50.0
411,412,2013.250,18.8,390.96960,7,24.97923,121.53986,40.6
412,413,2013.000,8.1,104.81010,5,24.96674,121.54067,52.5


In [2]:
df.describe()

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
count,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000
mean,207.500000,2013.148971,17.712560,1083.885689,4.094203,24.969030,121.533361,37.980193
std,119.655756,0.281967,11.392485,1262.109595,2.945562,0.012410,0.015347,13.606488
min,1.000000,2012.667000,0.000000,23.382840,0.000000,24.932070,121.473530,7.600000
25%,104.250000,2012.917000,9.025000,289.324800,1.000000,24.963000,121.528085,27.700000
50%,207.500000,2013.167000,16.100000,492.231300,4.000000,24.971100,121.538630,38.450000
75%,310.750000,2013.417000,28.150000,1454.279000,6.000000,24.977455,121.543305,46.600000
max,414.000000,2013.583000,43.800000,6488.021000,10.000000,25.014590,121.566270,117.500000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   No                                      414 non-null    int64  
 1   X1 transaction date                     414 non-null    float64
 2   X2 house age                            414 non-null    float64
 3   X3 distance to the nearest MRT station  414 non-null    float64
 4   X4 number of convenience stores         414 non-null    int64  
 5   X5 latitude                             414 non-null    float64
 6   X6 longitude                            414 non-null    float64
 7   Y house price of unit area              414 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 26.0 KB


In [4]:
cols = []
for col in df.columns:
    cols.append(col)
cols

['No',
 'X1 transaction date',
 'X2 house age',
 'X3 distance to the nearest MRT station',
 'X4 number of convenience stores',
 'X5 latitude',
 'X6 longitude',
 'Y house price of unit area']

Aineisto sisältää 414 havaintoa ja 8 saraketta. Yksi sarakkeista toimii havaintojen tunnisteena (No), kuusi saraketta sisältää numeerisia selittäviä muuttujia (X), jotka kuvaavat kiinteistöjen ominaisuuksia, ja yksi sarake toimii selitettävänä muuttujana (Y house price of unit area). Kohdemuuttujan arvot vaihtelevat välillä 7–117. Aineistossa ei ole puuttuvia arvoja (NaN/Null), ja kaikki muuttujat ovat numeerista tyyppiä.

# 3. Data Preparation

Tässä vaiheessa valmistellaan aineisto lineaarisen ja logistisen regressiomallin rakentamista varten. Poistamme tarpeettoman sarakkeen (No), muodostamme selittävien muuttujien matriisin (X) ja kohdemuuttujan (y), jaamme aineiston koulutus- ja testijoukkoon sekä muunnetaan kohdemuuttuja binääriseen muotoon logistista regressiota varten.

Tämän vaiheen päätavoitteena on tuottaa asianmukaisesti esikäsitelty aineisto, joka soveltuu mallien kouluttamiseen ja niiden objektiiviseen arviointiin.

Sarakkeesta No luovutaan, koska se toimii ainoastaan havaintojen tunnisteena eikä sisällä ennustamisen kannalta merkityksellistä informaatiota.

In [5]:
df = df.drop(columns=['No'])
df

,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1
...,...,...,...,...,...,...,...
409,2013.000,13.7,4082.01500,0,24.94155,121.50381,15.4
410,2012.667,5.6,90.45606,9,24.97433,121.54310,50.0
411,2013.250,18.8,390.96960,7,24.97923,121.53986,40.6
412,2013.000,8.1,104.81010,5,24.96674,121.54067,52.5


Seuraavaksi muodostetaan selittävien muuttujien matriisi (X) ja kohdemuuttuja (y). Tässä tapauksessa selittävinä muuttujina käytetään kuutta saraketta, jotka kuvaavat kiinteistöjen ominaisuuksia, ja kohdemuuttujana toimii “Y house price of unit area”.

In [6]:
X = df.drop(columns='Y house price of unit area')
y = df['Y house price of unit area']

Aineisto jaetaan tämän jälkeen koulutusjoukkoon (train) ja testijoukkoon (test) suhteessa 70% / 30%.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42
)

Logistisen regression soveltamiseksi jatkuva kohdemuuttuja muunnetaan binääriseen muotoon. Kiinteistöt luokitellaan luokkaan ”1”, jos niiden neliömetrihinta ylittää keskiarvon, ja luokkaan ”0”, jos hinta on keskiarvoa pienempi.

In [8]:
mean_price = y_train.mean()

y_train_binar = y_train.map(lambda x: 1 if x > mean_price else 0)
y_test_binar = y_test.map(lambda x: 1 if x > mean_price else 0)

Koska selittävillä muuttujilla on eri mittakaavat ja yksiköt, aineisto skaalataan. Skaalaus on erityisen tärkeää logistisessa regressiossa, sillä se parantaa optimointialgoritmin konvergenssia ja mahdollistaa mallin kertoimien luotettavamman tulkinnan.

In [9]:
scaler = StandardScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

Tässä vaiheessa suoritettiin kaikki tarvittavat esikäsittelytoimenpiteet lineaarisen ja logistisen regression soveltamista varten. Epärelevantit muuttujat (No) poistettiin, aineisto jaettiin koulutus- ja testijoukkoon, ja kohdemuuttujat valmisteltiin molempia malleja varten.

Aineiston jakaminen koulutus- ja testijoukkoon tehtiin mallien yleistämiskyvyn objektiivisen arvioinnin varmistamiseksi. Kaikki muunnosparametrit (skaalaus ja binäärisen luokittelun kynnysarvo) laskettiin yksinomaan koulutusjoukon perusteella.

# 4. Modeling

Tässä vaiheessa rakennetaan lineaarisen ja logistisen regressiomallin. Lineaarista regressiota käytetään neliömetrihinnan ennustamiseen, kun taas logistista regressiota sovelletaan kiinteistöjen luokitteluun binäärisen kohdemuuttujan perusteella.

## 4.1. Linear Regression

Lineaarista regressiota käytetään mallintamaan jatkuvan kohdemuuttujan ja numeeristen selittävien muuttujien välistä riippuvuutta. Malli arvioi kunkin muuttujan vaikutusta kiinteistön neliömetrihintaan.

Tässä vaiheessa rakennetaan ja koulutetaan lineaarinen regressiomalli.

In [10]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


Seuraavaksi tuotetaan ennusteet mallin avulla.

In [11]:
y_pred_train = model.predict(X_train_scaled)

Lopuksi tarkastellaan mallin kertoimia ja vakiotermiä (intercept).

In [12]:
model_coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
})

print(model_coefficients)
print(f'Intercept: %.2f' % model.intercept_)

                                  Feature  Coefficient
0                     X1 transaction date     1.651220
1                            X2 house age    -2.760935
2  X3 distance to the nearest MRT station    -6.404436
3         X4 number of convenience stores     3.205465
4                             X5 latitude     2.989293
5                            X6 longitude    -0.787719
Intercept: 38.44


Intercept-arvo kuvaa ennustettua hintaa tilanteessa, jossa kaikki selittävät muuttujat ovat keskimääräisellä tasolla, ja se on lähellä kohdemuuttujan keskiarvoa koulutusjoukossa. Mallin kertoimet osoittavat, kuinka paljon hinta muuttuu, kun kyseinen muuttuja poikkeaa keskiarvostaan yhden keskihajonnan verran.

Kertoimen positiivinen merkki tarkoittaa, että hinnan ennustetaan kasvavan kyseisen muuttujan arvon kasvaessa, kun taas negatiivinen merkki tarkoittaa hinnan laskua muiden tekijöiden pysyessä ennallaan.

Tulosten perusteella suurin vaikutus hinnan ennustamiseen on etäisyydellä lähimpään metroasemaan (X3 - mitä suurempi etäisyys, sitä alhaisempi hinta), kun taas pienin vaikutus on muuttujalla longitude (X6). Muuttujien X4 (lähikauppojen määrä) ja X5 (latitude) positiiviset kertoimet viittaavat siihen, että niiden arvon kasvu on yhteydessä korkeampaan hintaan.

## 4.2. Logistic Regression

Logistista regressiota käytetään binäärisiin luokittelutehtäviin. Tässä työssä sitä sovelletaan määrittämään, kuuluuko kiinteistö keskiarvoa korkeampien vai matalampien hintojen luokkaan binääriseksi muunnetun kohdemuuttujan perusteella.

Tässä vaiheessa rakennetaan ja koulutetaan logistinen regressiomalli.

In [13]:
model2 = LogisticRegression(solver='lbfgs')
model2.fit(X_train_scaled, y_train_binar)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


Mallin avulla ennustetaan luokat (0/1).

In [14]:
y_pred_train2 = model2.predict(X_train_scaled)

Lopuksi tarkastellaan mallin kertoimia ja vakiotermiä.

In [15]:
model2_coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model2.coef_[0]
})

print(model2_coefficients)
print(f'Intercept: %.2f' % model2.intercept_)

                                  Feature  Coefficient
0                     X1 transaction date     0.169401
1                            X2 house age    -0.593532
2  X3 distance to the nearest MRT station    -2.470506
3         X4 number of convenience stores     0.579035
4                             X5 latitude     1.111065
5                            X6 longitude    -0.072731
Intercept: -0.59


C:\Users\onion\AppData\Local\Temp\ipykernel_1128\76991883.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f'Intercept: %.2f' % model2.intercept_)


Logistisen regression kertoimien analyysi osoittaa, että etäisyys lähimpään metroasemaan (X3) ja rakennuksen ikä (X2) pienentävät todennäköisyyttä kuulua keskiarvoa korkeampien hintojen luokkaan. Sen sijaan lähikauppojen määrä (X4) ja leveysaste (X5) vaikuttavat positiivisesti tähän todennäköisyyteen. Muuttujalla longitude (X6) on pienin vaikutus mallissa.

Koska selittävät muuttujat standardoitiin ennen mallin kouluttamista, logistisen regression kertoimet ovat keskenään vertailukelpoisessa mittakaavassa ja niitä voidaan käyttää muuttujien suhteellisen tärkeyden arviointiin.

# 5. Evaluation

Tässä vaiheessa arvioidaan rakennettujen mallien suorituskyky testijoukolla, jota ei käytetty mallien kouluttamiseen. Lineaarisen ja logistisen regression arvioinnissa käytetään metriikoita, joiden avulla voidaan tarkastella ennusteiden tarkkuutta ja mallien yleistämiskykyä. Saadut tulokset analysoidaan ja vertaillaan.

## 5.1. Linear Regression

Lineaarisen regression suorituskykyä arvioidaan käyttämällä Mean Absolute Error (MAE) -metriikkaa, joka lasketaan testijoukolla. MAE kuvaa mallin ennusteiden keskimääräistä absoluuttista poikkeamaa todellisista arvoista.

In [16]:
y_pred_test = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred_test)

print(f'Mean Absolute Error: %.2f' % mae)

Mean Absolute Error: 6.18


Tässä työssä MAE-arvo testijoukolla oli 6.18.

## 5.2. Logistic Regression

Logistisen regression arviointi suoritetaan testijoukolla mallin yleistämiskyvyn tarkastelemiseksi.

In [17]:
y_pred_test2 = model2.predict(X_test_scaled)

acc = accuracy_score(y_test_binar, y_pred_test2)
cm = confusion_matrix(y_test_binar, y_pred_test2)


print(f'Accuracy: %.2f' % acc)
print(cm)

Accuracy: 0.81
[[52 20]
 [ 4 49]]


Accuracy-arvo testijoukolla oli 0.81. Sekaannusmatriisi osoittaa, että mallissa esiintyy sekä vääriä positiivisia että vääriä negatiivisia luokitteluja.

# 6. Deployment

Tässä työssä rakennettiin ja analysoitiin lineaarisen ja logistisen regression mallit kiinteistöjen hintojen arvioimiseksi numeeristen selittävien muuttujien perusteella. Työssä noudatettiin CRISP-DM-prosessimallia, ja sen mukaisesti toteutettiin aineiston analyysi, esikäsittely, mallinnus sekä mallien suorituskyvyn arviointi.

Lineaarista regressiota käytettiin jatkuvan kohdemuuttujan, eli neliömetrihinnan, ennustamiseen. Mallin arviointi testijoukolla tuotti Mean Absolute Error -arvon 6.18, mikä kuvaa mallin ennusteiden keskimääräistä absoluuttista poikkeamaa todellisista arvoista. Kertoimien analyysi osoitti, että suurin vaikutus kiinteistön hintaan on etäisyydellä lähimpään metroasemaan, lähikauppojen määrällä sekä kiinteistön maantieteellisellä sijainnilla (latitude).

Logistista regressiota sovellettiin binääriseen luokitteluun sen perusteella, ylittääkö kiinteistön neliömetrihinta keskiarvon. Testijoukolla saavutettu accuracy-arvo oli 0.81. Sekaannusmatriisi osoittaa, että malli erottaa useimmissa tapauksissa oikein keskiarvoa korkeammat ja matalammat hinnat, vaikka joissakin tapauksissa esiintyy virheellisiä luokituksia.

Kokonaisuutena tulokset osoittavat, että molemmat mallit pystyvät hyödyntämään selittäviä muuttujia tehokkaasti. Lineaarinen regressio mahdollistaa kiinteistöjen hintojen kvantitatiivisen ennustamisen, kun taas logistinen regressio ratkaisee luokittelutehtävän saman aineiston perusteella. Tulokset osoittavat, että tarkastellut menetelmät soveltuvat kiinteistöjen hintojen analysointiin.